In this file, the goal to create a .csv file containing the anomaly score (using feature bagging with One-Class SVM) for each subject in PEPR_Marseille (including those who were not selected) as the file containing the reconstruction_error

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.stats import ks_2samp
from scipy import stats
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

In [2]:

def read_error_csv(path):

    if os.path.exists(path):
        df = pd.read_csv(path)
        return df
    return None

In [3]:
def remove_first_column(Embeddings):
    X= Embeddings.iloc[:,1:76]
    return X

In [4]:
def Normalisation(X1, X4):
    scaler = StandardScaler()
    X1 = scaler.fit_transform(X1)

    X4=scaler.transform(X4)

    return X1, X4

In [5]:
dir_path= "/neurospin/tmp/fred/models/2025-11-10" 

In [6]:
import re

Calcul du score d'anomalies

In [7]:
def Calcul_score(X1, X4, nu=0.02):


    n_models = 25
    M=75
    subset_size= M//2

    #score_all1 = np.zeros((X1.shape[0], n_models))

    score_all4 = np.zeros((X4.shape[0], n_models))

    for j in range(n_models):
                # Sélection aléatoire d'un sous-ensemble de features
        features_idx = np.random.choice(M, subset_size, replace=False)
        X_subset1 = X1[:, features_idx]

        X_subset4 = X4[:, features_idx]

        clf = OneClassSVM(kernel='rbf', gamma='scale', nu=nu)
        clf.fit(X_subset1)
        #score1 = clf.decision_function(X_subset1)

        score4 = clf.decision_function(X_subset4)

        #score_all1[:, j] = score1

        score_all4[:, j] = score4


    return  -np.mean(score_all4, axis=1)

In [8]:
import re


In [9]:
subfolders = sorted([
    f for f in os.listdir(dir_path)
    if os.path.isdir(os.path.join(dir_path, f))
])



for i, folder in enumerate(subfolders):
    print(i)
    regions=re.match(r"(.*?(left|right))", folder).group(1)

    current_path = os.path.join(dir_path, folder)

    ukb_file = os.path.join(current_path, "Embeddings.csv")

    PEPR_file= os.path.join(current_path, "PEPR_Marseille","Embeddings.csv")
 
    current_path = os.path.join(dir_path, folder, "PEPR_Marseille")

    # lecture
    Embeddings_ukb = read_error_csv(ukb_file)

    Embeddings_PEPR = read_error_csv(PEPR_file)



    X1= remove_first_column(Embeddings_ukb)

    X4= remove_first_column(Embeddings_PEPR)

    X1, X4= Normalisation(X1, X4)
    
    score_PEPR = Calcul_score(X1, X4)

    dico_score= {"ID": Embeddings_PEPR.iloc[:,0],
                 "Score": score_PEPR}
    df_score= pd.DataFrame(dico_score)
    df_score.to_csv(os.path.join(current_path, "Score_Bagging_SVM.csv"))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
